##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rede Neural Convolucional (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/images/cnn.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/images/cnn.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este tutorial demonstra o treinamento de uma [Rede Neural Convolucional](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN, na sigla em inglês) simples para classificar [imagens do CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html). Como este tutorial usa a [API Sequential do Keras](https://www.tensorflow.org/guide/keras/overview), serão necessárias apenas algumas linhas de código para criar e treinar o modelo.


### Importar o TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Baixar e preparar o dataset CIFAR10

O dataset CIFAR10 contém 60.000 imagens coloridas em 10 classes, com 6.000 imagens em cada uma. O dataset é dividido em 50.000 imagens de treinamento e 10.000 imagens de teste. As classes são mutuamente exclusivas, e não há sobreposições entre elas.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verificar os dados

Para verificar se o dataset parece estar correto, vamos plotar as primeiras 25 imagens do conjunto de treinamento e exibir o nome da classe abaixo de cada imagem:


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Criar a base convolucional

As seis linhas de código abaixo definem a base convolucional usando um padrão comum: uma pilha de camadas [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) e [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

Uma CNN recebe como entrada tensores de formato (image_height, image_width, color_channels), ignorando o tamanho do lote. Se essas dimensões forem novidade para você, color_channels refere-se a (R,G,B). Neste exemplo, você vai configurar sua CNN para processar entradas de formato (32, 32, 3), que é o formato das imagens do CIFAR. Para fazer isso, basta passar o argumento `input_shape` para sua primeira camada.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Vamos exibir a arquitetura do modelo até o momento:

In [ ]:
model.summary()

Acima, você pode ver que a saída de cada camada Conv2D e MaxPooling2D é um tensor tridimensional de formato (altura, largura, canais). As dimensões de largura e altura costumam encolher à medida que você se aprofunda na rede. O número de canais de saída para cada camada Conv2D é controlado pelo primeiro argumento (por exemplo, 32 ou 64). Tipicamente, como a largura e a altura encolhem, você consegue suportar (computacionalmente) a adição de mais camadas de saída a cada camada Conv2D.

### Adicionar camadas Dense

Para completar o modelo, você alimentará o último tensor de saída da base convolucional [de formato (4, 4, 64)] em uma ou mais camadas Dense para fazer a classificação. As camadas Dense recebem vetores como entrada (com 1 dimensão), enquanto a saída atual é um tensor com 3 dimensões. Primeiro, você achatará (ou converterá) a saída de 3 dimensões para 1 dimensão, depois adicionará uma ou mais camadas Dense. O CIFAR tem 10 classes de saída, então você usa uma camada Dense final com 10 saídas.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

Veja a arquitetura completa do modelo:

In [ ]:
model.summary()

O resumo da rede mostra que saídas (4, 4, 64) foram achatadas em vetores de formato (1024) antes de passarem por duas camadas Dense.

### Compilar e treinar o modelo

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Avaliar o modelo

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

A sua CNN simples alcançou uma exatidão de teste acima de 70%. Nada mal para poucas linha de código! Para ver outro estilo de CNN, confira o exemplo [Guia de início rápido do TensorFlow 2 para especialistas](https://www.tensorflow.org/tutorials/quickstart/advanced), que usa a API de subclasses do Keras e `tf.GradientTape`.